In [1]:
import numpy as np
from transformers import AutoModelForCausalLM
import openvino as ov

import torch

model_vendor, model_name = 'stabilityai', 'japanese-stablelm-base-alpha-7b'

"""
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple, CausalLMOutputWithPast]:
"""

C:\Users\yshimur1\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


'\n        input_ids: Optional[torch.LongTensor] = None,\n        attention_mask: Optional[torch.FloatTensor] = None,\n        position_ids: Optional[torch.LongTensor] = None,\n        inputs_embeds: Optional[torch.FloatTensor] = None,\n        head_mask: Optional[torch.FloatTensor] = None,\n        past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None,\n        labels: Optional[torch.LongTensor] = None,\n        use_cache: Optional[bool] = None,\n        output_attentions: Optional[bool] = None,\n        output_hidden_states: Optional[bool] = None,\n        return_dict: Optional[bool] = None,\n    ) -> Union[Tuple, CausalLMOutputWithPast]:\n'

In [2]:
model = AutoModelForCausalLM.from_pretrained(f'{model_vendor}/{model_name}', trust_remote_code=True)
model.eval()

C:\Users\yshimur1\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
C:\Users\yshimur1\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

JapaneseStableLMAlphaForCausalLM(
  (transformer): JapaneseStableLMAlphaModel(
    (embed_in): Embedding(65536, 4096)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=False)
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): Attention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): Linear(in_features=4096, out_features=12288, bias=False)
          (dense): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MLP(
          (packed_input_proj): Linear(in_features=4096, out_features=22016, bias=False)
          (out_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act): SiLU()
        )
      )
    )
    (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (embed_out): Linear(in_features=4096, out_features=65536, bias=False)
)

In [3]:
num_seq = 100
n = num_seq
# from config.json
vocab_size = 65536
hidden_size = 4096
num_hidden_layers = 32
num_attention_heads = 32

example_input = {
    'input_ids': torch.tensor([[123]], dtype=torch.int),
    'attention_mask': torch.tensor([[1]],dtype=torch.int),
    'position_ids': torch.tensor([[0]], dtype=torch.int),
    #'inputs_embeds': np.array([0], dtype=np.int32),
    # input head_mask has shape [num_heads] or [num_hidden_layers x num_heads]
    #'head_mask': torch.Tensor(size=(num_attention_heads,1,1,1,n)).to(torch.float32),

    'past_key_values': torch.Tensor(size=(num_hidden_layers, 2, 1, num_attention_heads, num_seq, hidden_size // num_hidden_layers)).to(torch.float32),
    #'past_key_values': torch.Tensor(size=(1, 2, 1, num_attention_heads, num_seq, hidden_size // num_hidden_layers)).to(torch.float32),
    #  past_key_values [n][0|1][ 1, 32, seq_len, 128]

    #'labels': np.zeros((1,100), dtype=np.int32),
    'use_cache': torch.tensor([1], dtype=torch.int32),
    'output_attentions': torch.tensor([32], dtype=torch.int32),
    'output_hidden_states': torch.tensor([32], dtype=torch.int32),
    'return_dict': torch.tensor([0], dtype=torch.int32),
}

#print(example_input)

In [7]:
from nncf import compress_weights, CompressWeightsMode
import nncf

In [11]:
ov_model = ov.convert_model(model, example_input=example_input)
print(ov_model)

<Model: 'Model2'
inputs[
<ConstOutput: names[input_ids] shape[?,?] type: i32>,
<ConstOutput: names[attention_mask] shape[?,?] type: i32>,
<ConstOutput: names[position_ids] shape[?,?] type: i32>,
<ConstOutput: names[past_key_values] shape[?,?,?,?,?,?] type: f32>,
<ConstOutput: names[use_cache] shape[?] type: i32>,
<ConstOutput: names[output_attentions] shape[?] type: i32>,
<ConstOutput: names[output_hidden_states] shape[?] type: i32>,
<ConstOutput: names[return_dict] shape[?] type: i32>
]
outputs[
<ConstOutput: names[7270, 76] shape[?,?,65536] type: f32>,
<ConstOutput: names[11, 189, hidden_states.1, 7205] shape[?,?,4096] type: f32>,
<ConstOutput: names[437, hidden_states.3, 439, 7206, 12] shape[?,?,4096] type: f32>,
<ConstOutput: names[655, 7207, 657, hidden_states.5, 13] shape[?,?,4096] type: f32>,
<ConstOutput: names[873, hidden_states.7, 875, 7208, 14] shape[?,?,4096] type: f32>,
<ConstOutput: names[1091, hidden_states.9, 1093, 7209, 15] shape[?,?,4096] type: f32>,
<ConstOutput: nam

In [12]:
#compressed_model = compress_weights(ov_model, mode=CompressWeightsMode.INT8_SYM)
compressed_model = compress_weights(ov_model, mode=CompressWeightsMode.INT4_ASYM)

INFO:nncf:Statistics of the bitwidth distribution:
+--------------+---------------------------+-----------------------------------+
| Num bits (N) | % all parameters (layers) |    % ratio-defining parameters    |
|              |                           |             (layers)              |
+==============+===========================+===================================+
| 8            | 8% (2 / 130)              | 0% (0 / 128)                      |
+--------------+---------------------------+-----------------------------------+
| 4            | 92% (128 / 130)           | 100% (128 / 128)                  |
+--------------+---------------------------+-----------------------------------+


Output()

In [13]:
ov.save_model(compressed_model, 'openvino_model_int4asym.xml')

In [ ]:
#from optimum.intel.openvino import OVModelForCausalLM
#model = OVModelForCausalLM.from_pretrained(f'{model_vendor}/{model_name}', trust_remote_code=True, export=True, compile=False, load_in_8bit=False)

"""
<Model: 'Model0'
inputs[
<ConstOutput: names[input_ids] shape[?,?] type: i32>,
<ConstOutput: names[attention_mask] shape[?,?] type: f32>,
<ConstOutput: names[position_ids] shape[?,?] type: i32>,
<ConstOutput: names[head_mask, 146, head_mask.3] shape[?,?,?,?,?] type: f32>,
<ConstOutput: names[use_cache] shape[?] type: i32>,
<ConstOutput: names[output_attentions] shape[?] type: i32>,
<ConstOutput: names[output_hidden_states] shape[?] type: i32>,
<ConstOutput: names[return_dict] shape[?] type: i32>
]
outputs[
<ConstOutput: names[6522] shape[?,?,65536] type: f32>
]>
"""